In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
#pip install category_encoders
import category_encoders as ce

from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , RandomizedSearchCV,KFold,RepeatedKFold

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import learning_curve


from matplotlib.colors import ListedColormap
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn import svm 
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.inspection import permutation_importance
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, RocCurveDisplay, DetCurveDisplay,PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score

#  №

In [ ]:
path_train = pd.read_csv('income_train.csv')
df = pd.DataFrame(path_train)
df['income'] = np.where(df['income']== '>50K',1,0)
df.head()

In [ ]:
ax, f = plt.subplots(figsize = (7,7))
df[['income']].value_counts().plot.pie()

In [ ]:
ax = sns.pairplot(df, hue = 'income', palette= 'inferno')

In [ ]:
ax = sns.regplot(x = 'age' , y = 'hours-per-week', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
ax = sns.regplot(x = 'hours-per-week' , y = 'capital-loss', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
df['capital-gain'].unique().max()

In [ ]:
df[df['capital-gain'] == 99999]

In [ ]:
ax = sns.regplot(x = 'age' , y = 'capital-gain', data = df, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
df.workclass.value_counts()

In [ ]:
df[df.workclass=='?']

In [ ]:
df = df.replace('?',np.NaN)
df = df.dropna()
# это выброс, тк рядом значений нет, а самое близкое к этому это где-то 43000
df = df.drop(df[df['capital-gain'] == 99999].index)
df.head()

In [ ]:
ax = sns.pairplot(df, hue = 'income', palette= 'inferno')

In [ ]:
ax, f = plt.subplots(figsize = (5,5))
sns.barplot(data = df, x = 'income', y = 'hours-per-week', hue = 'occupation', palette= 'inferno')

In [ ]:
sns.histplot(df,x= 'hours-per-week', bins = 10, hue = 'sex', palette='inferno')

In [ ]:
df.isna().sum()

# Block of category changes 

## Relationship

In [ ]:
df.relationship.unique()

In [ ]:
plt.figure(figsize=(17,10))
total = float(len(df))

ax = sns.countplot(x="relationship", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
df.relationship = df.relationship.replace('Husband','Family')
df.relationship = df.relationship.replace('Wife','Family')
df.relationship = df.relationship.replace('Unmarried','Not-in-family')
df.relationship = df.relationship.replace('Own-child','Other')
df.relationship = df.relationship.replace('Other-relative','Other')



## Occupation

In [ ]:
plt.figure(figsize=(17,10))
total = float(len(df))

ax = sns.countplot(x="occupation", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
df[df.occupation=='?'].head()

In [ ]:
# occupation разделим группы работников на разные категории
# Intellectual labor и Physical labor - работают почасовая оплата или месячная
# а также применение разных способностей на работе(сила/ум)
df.occupation = df.occupation.replace('Prof-specialty','Professional')
df.occupation = df.occupation.replace('Craft-repair','Physical labor') # blue - рабочий класс(почасовая полата) - физич труд
df.occupation = df.occupation.replace('Exec-managerial','Intellectual labor') # white - наемный работник(доход в месяц) - умственный
df.occupation = df.occupation.replace('Adm-clerical','Intellectual labor') # white
df.occupation = df.occupation.replace('Machine-op-inspct','Physical labor') # blue
df.occupation = df.occupation.replace('Transport-moving','Physical labor')# blue
df.occupation = df.occupation.replace('Handlers-cleaners','Physical labor') # blue
df.occupation = df.occupation.replace('Farming-fishing','Physical labor')# blue
df.occupation = df.occupation.replace('Tech-support','Other-service')
df.occupation = df.occupation.replace('Protective-serv','Other-service')
df.occupation = df.occupation.replace('Priv-house-serv','Other-service')
df.occupation = df.occupation.replace('Armed-Forces','Other-service')




## Workclass

In [ ]:
plt.figure(figsize=(17,10))
total = float(len(df))

ax = sns.countplot(x="workclass", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
# workclass
df.workclass = df.workclass.replace('Federal-gov','Government')
df.workclass = df.workclass.replace('State-gov','Government')
df.workclass = df.workclass.replace('Local-gov','Government')
df.workclass = df.workclass.replace('Self-emp-inc','Self-Employed')
df.workclass = df.workclass.replace('Self-emp-not-inc','Self-Employed')
df.workclass = df.workclass.replace('Never-worked', 'Other')
df.workclass = df.workclass.replace('Without-pay', 'Other')



## Marital-status

In [ ]:
plt.figure(figsize=(17,10))
total = float(len(df))

ax = sns.countplot(x="marital-status", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
df['marital-status'].value_counts()

In [ ]:
#marital status сделаем меньше категорий, 
#тк замужних можно объеденить в 1 категорию и никогда не замужних поменяем на одиноких
df['marital-status'] = df['marital-status'].replace('Married-civ-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-AF-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-spouse-absent','Married')
df['marital-status'] = df['marital-status'].replace('Never-married','Single')

#df['marital-status'] = df['marital-status'].replace('Divorced','Single')
#df['marital-status'] = df['marital-status'].replace('Widowed','Single')
#df['marital-status'] = df['marital-status'].replace('Separated','Single')

## Race

In [ ]:
plt.figure(figsize=(17,10))
total = float(len(df))

ax = sns.countplot(x="race", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
display(pd.DataFrame(df[['race']].value_counts()))
# можно сделать как белые и остальные

In [ ]:
df.race = df.race.replace('Black','Other')
df.race = df.race.replace('Asian-Pac-Islander','Other')
df.race = df.race.replace('Amer-Indian-Eskimo','Other')

## Country

In [ ]:
country = df['native-country'].unique()

In [ ]:
def zamena_country(data):
    for i in country:
        if i != 'United-States':
            data['native-country'] = data['native-country'].replace(i,'Other')
    return data

In [ ]:
df = zamena_country(df)

In [ ]:
df.head()

In [ ]:
df['native-country'].unique()

## Можно сделать умно и из 2 столбцов сделать один(как изменение дохода от инвестиций)

In [ ]:
df['capital-changes']  = df['capital-gain'] - df['capital-loss']
#df['capital-changes'].describe()
# и также дропнуть 2 столбца, которые мы здесь использовали, тк все показали в одном
# но дропнем в блоке ниже

In [ ]:
df["capital-change"].hist(figsize=(8,8))
plt.show()

In [ ]:
df.head()

## Education

In [ ]:
plt.figure(figsize=(20,10))
total = float(len(df["income"]) )

ax = sns.countplot(x="education", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
#df.education = df.education.replace(['Preschool','1st-4th','5th-6th','7th-8th','9th','10th','11th','12th','HS-grad'],'School')
#df.education = df.education.replace(['Bachelors','Some-college','Prof-school','Assoc-voc'],'Higher-educ')
#df.education = df.education.replace(['Assoc-acdm','Masters','Doctorate'],'Academia')


df.education = df.education.replace(['Preschool','1st-4th','5th-6th','7th-8th','9th','10th','11th','12th'],'School')
df.education = df.education.replace(['Bachelors','Some-college','Masters','Prof-school','Doctorate','HS-grad'],'Higher-educ')
df.education = df.education.replace(['Assoc-acdm','Assoc-voc'],'Cources')

### это ухудшает результаты поэтому не использовал 

In [ ]:
plt.figure(figsize=(20,10))
total = float(len(df["income"]) )

ax = sns.countplot(x="education-num", hue="income", data=df)
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:1.2f}'.format((height/total)*100),
            ha="center") 
plt.show()

In [ ]:
df[['education','education-num']].value_counts()
# assoc-voc это как дополнительноен образование(курсы), где потом дается диплом, что закончил вот это то то

In [ ]:
df = df.drop(columns=['education'])

## Hours

In [ ]:
ax, f = plt.subplots(figsize = (15,5))
sns.histplot(data=df, x = 'hours-per-week', hue='income' , palette='YlOrBr')

# видно, что люди 40-60 получают больше, чем остальные группы

In [ ]:
df.head()

## Удаление ненужных столбцов после обработки категор данных

In [ ]:
df = df.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
df.head()

# Column Transformer

In [ ]:
# доп трансф для изменений

workclass_map = [{
    'col':'workclass', 
    'mapping':{ 'Other':3, 'Government':1, 'Self-Employed':2, 'Private':0}
    }]

marital_map = [{
    'col':'marital-status',
    'mapping':{'Single':1, 'Married':0}
    }]

occupation_map = [{
    'col':'occupation',
    'mapping':{'Other-service':4, 'Sales':3,'Physical labor':2, 'Professional':1, 'Intellectual labor':0}
    }]
education_map = [{
    'col':'education',
    'mapping':{'School':2, 'Cources':1, 'Higher-educ':0}
    }]
relationship_map = [{
    'col':'relationship',
    'mapping':{'Not-in-family':1, 'Family':0}
    }]
native_country_map = [{
    'col':'native-country',
    'mapping':{'Other':1, 'United-States':0}
    }]
sex_map = [{
    'col':'sex',
    'mapping':{'Female':1, 'Male':0}
    }]
race_map = [{
    'col':'race',
    'mapping':{'Other':1, 'White':0}
    }]
sex_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = sex_map))])

workclass_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = workclass_map))])

marital_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = marital_map))])

occupation_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = occupation_map))])

race_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = race_map))])

native_country_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = native_country_map))])
education_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = education_map))])
relationship_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = relationship_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('sex', sex_transformer, ['sex']),
                                     ('work', workclass_transformer, ['workclass']),
                                     ('matiral', marital_transformer, ['marital-status']),
                                     ('occupation', occupation_transformer, ['occupation']),
                                     ('race', race_transformer, ['race']),
                                     ('native-country',native_country_transformer, ['native-country']),
                                     ('education',education_transformer, ['education']),
                                     ('relationship',relationship_transformer,['relationship'])],
                                      remainder = 'passthrough')

##display(CT)

#X_CT = CT.fit_transform(X)
#display(pd.DataFrame(X_CT))

In [ ]:
df.head()

### Main column transf

In [ ]:
num_features = ['age','hours-per-week','capital-changes']
y = np.array(df.income)
X = df.drop(columns=['income'])

In [ ]:

workclass_map = [{
    'col':'workclass', 
    'mapping':{ 'Other':3, 'Government':2, 'Self-Employed':1, 'Private':0}
    }]
occupation_map = [{
    'col':'occupation',
    'mapping':{'Other-service':4, 'Sales':3,'Physical labor':2, 'Professional':1, 'Intellectual labor':0}
    }]
marital_map = [{
    'col':'marital-status',
    'mapping':{'Separated': 4, 'Widowed': 3, 'Divorced':2, 'Single': 1, 'Married':0}
    }]
sex_map = [{
    'col':'sex',
    'mapping':{'Female':1, 'Male':0}
    }]
race_map = [{
    'col':'race',
    'mapping':{'Other':1, 'White':0}
    }]
native_country_map = [{
    'col':'native-country',
    'mapping':{'Other':1, 'United-States':0}
    }]
education_map = [{
    'col':'education',
    'mapping':{'Academia':2, 'School':1, 'Higher-educ':0}
    }]
relationship_map = [{
    'col':'relationship',
    'mapping':{'Other':2, 'Not-in-family':1, 'Family':0}
    }]

sex_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = sex_map))])

workclass_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = workclass_map))])

marital_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = marital_map))])

occupation_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = occupation_map))])

race_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = race_map))])

native_country_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = native_country_map))])
education_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = education_map))])
relationship_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = relationship_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('sex', sex_transformer, ['sex']),
                                     ('work', workclass_transformer, ['workclass']),
                                     ('matiral', marital_transformer, ['marital-status']),
                                     ('occupation', occupation_transformer, ['occupation']),
                                     ('race', race_transformer, ['race']),
                                     ('native-country',native_country_transformer, ['native-country']),
                                     ('relationship',relationship_transformer,['relationship'])],
                                      remainder = 'passthrough')

display(CT)

X_CT = CT.fit_transform(X)
display(pd.DataFrame(X_CT))

# Features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_CT, y)

In [ ]:
def plot_learnig_tree(estimator, X_train, X_test, y_train, y_test):  
    
    fig = plt.figure(figsize=(20,5))
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    n_estimators = estimator.n_estimators_
    zero = np.zeros(n_estimators, dtype=np.float64)
    color_one = 'blue'
    color_two = 'red'
    
    train_score = zero
    for i, y_pred_train in enumerate(estimator.staged_predict(X_train)):
        train_score[i] = accuracy_score(y_train, y_pred_train)
    ax1.plot(
        (np.arange(train_score.shape[0]) + 1),
        train_score,
        "-",
        color=color_one,
        label='Train',
        )
        
    test_score = zero
    for i, y_pred_test in enumerate(estimator.staged_predict(X_test)):
        train_score[i] = accuracy_score(y_test, y_pred_test)    
    ax1.plot(
        (np.arange(test_score.shape[0]) + 1),
        train_score,
        "-",
        color = color_two,
        label='Test',
        )
    ax1.legend(loc='best')


    train_deviance = zero
    for i, y_pred_train in enumerate(estimator.staged_decision_function(X_train)):
        train_deviance[i] = estimator.loss_(y_train, y_pred_train)
        
    ax2.plot(
        (np.arange(train_deviance.shape[0]) + 1),
        train_deviance,
        '-',
        color=color_one,
        label="score "+str(round(estimator.score(X_train, y_train),3))
    )

    test_deviance = zero
    for i, y_pred_test in enumerate(estimator.staged_decision_function(X_test)):
        test_deviance[i] = estimator.loss_(y_test, y_pred_test)
        
    ax2.plot(
        (np.arange(test_deviance.shape[0]) + 1),
        test_deviance,
        '-',
        color=color_two,
        label="score "+str(round(estimator.score(X_test, y_test),3))
    )
    ax2.legend(loc='best')
    

clf = GradientBoostingClassifier(n_estimators = 500,random_state=42,learning_rate = 0.09)
clf.fit(X_train, y_train)
plot_learnig_tree(clf, X_train, X_test, y_train, y_test)

clf = GradientBoostingClassifier(n_estimators=6000,random_state=42,learning_rate=0.95)
clf.fit(X_train, y_train)
plot_learnig_tree(clf, X_train, X_test, y_train, y_test)


In [ ]:
clf = GradientBoostingClassifier(n_estimators = 10000,learning_rate = 0.95)
clf.fit(X_CT, y)
clf.score(X_CT,y)

In [ ]:
original_params = {
    "n_estimators": 100,
    "random_state": 42,
    'criterion': 'friedman_mse'
    
}

fig = plt.figure(figsize=(20,5))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132)
ax3 = fig.add_subplot(133)

for label, color, setting in [
    ("Полный шаг", "red", {"learning_rate": 1, "subsample": 1.0}),
    ("Шаг 0.5", "orange", {"learning_rate": 0.5, "subsample": 1.0}),
    ("Шаг 0.1", "blue",   {"learning_rate": 0.1, "subsample": 1.0}),
    ("Шаг 0.1, фичи=sqrt",
              "lightblue", {"learning_rate": 0.1, "subsample": 1.0, "max_features": 'sqrt'},
    ),
    ("learning_rate=0.1, лист до 5",
               "darkblue", {"learning_rate": 0.1, "subsample": 1.0, "min_samples_split" : 5},
    ),
    ("learning_rate=0.1, баггинг=0.75",
                  "green", {"learning_rate": 0.1, "subsample": 0.75},
    ),
    ("learning_rate=0.1, баггинг=0.75, лист до 5",
             "lightgreen", {"learning_rate": 0.1, "subsample": 0.75, "min_samples_split" : 5},
    ),
    ("learning_rate=0.01, баггинг=0.5, лист до 5, фичи-часть",
             "black", {"learning_rate": 0.01, "subsample": 0.5, "min_samples_split" : 5, "max_features": 'sqrt'},
    ),
]:
    params = dict(original_params)
    params.update(setting)

    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
    
    train_deviance = np.zeros((params["n_estimators"],), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_train)):
        train_deviance[i] = clf.loss_(y_train, y_pred)
        
    ax1.plot(
        (np.arange(train_deviance.shape[0]) + 1),
        train_deviance,
        "-",
        color=color,
        label=label,
    )

    test_score = np.zeros((params["n_estimators"],), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_predict(X_test)):
        #clf.loss_ assumes that y_test[i] in {0, 1}
        test_score[i] = accuracy_score(y_test, y_pred)
        
    ax2.plot(
        (np.arange(test_score.shape[0]) + 1),
        test_score,
        "-",
        color=color,
        label=label,
    )
    
    test_deviance = np.zeros((params["n_estimators"],), dtype=np.float64)
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        #clf.loss_ assumes that y_test[i] in {0, 1}
        test_deviance[i] = clf.loss_(y_test, y_pred)
        

    ax3.plot(
        (np.arange(test_deviance.shape[0]) + 1),
        test_deviance,
        "-",
        color=color,
        label=label,
    )

#ax2.legend(loc='lower center')
ax2.set_xlabel("Boosting Iterations")
ax2.set_ylim(0.8, 1.)
ax1.set_ylabel("Train Set Deviance")
ax2.set_ylabel("Accuracy")
ax3.set_ylabel("Test Set Deviance")

plt.show()

In [ ]:
gbrt = GradientBoostingClassifier(learning_rate= 0.95 , subsample=0.75, n_estimators=500)
gbrt.fit(X_CT, y)
y_pred = gbrt.predict(X_CT)

ConfusionMatrixDisplay(confusion_matrix(y,y_pred)).plot();

In [ ]:
rf = RandomForestClassifier(n_estimators = 70)
rf.fit(X_CT, y)
y_pred = rf.predict(X_CT)

ConfusionMatrixDisplay(confusion_matrix(y,y_pred)).plot();

In [ ]:
RocCurveDisplay.from_predictions(y, y_pred);
DetCurveDisplay.from_predictions(y, y_pred);
PrecisionRecallDisplay.from_predictions(y, y_pred);

In [ ]:
pd.DataFrame(X_CT)

# Модели для примерного отбора

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_CT,y)
rf.score(X_CT,y)

In [ ]:
rff = RandomForestClassifier(n_estimators=1050)
rff.fit(X_CT,y)
rff.score(X_CT,y)

In [ ]:
y_predict = rf.predict(X_CT)

In [ ]:
roc_auc_score(y,y_predict)

In [ ]:
ada = AdaBoostClassifier(base_estimator = RandomForestClassifier(n_estimators = 200))
ada.fit(X_CT,y)
ada.score(X_CT,y)

In [ ]:
y_predict = ada.predict(X)

In [ ]:
accuracy_score(y,y_predict)

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_CT,y)
clf.score(X_CT,y)

In [ ]:
y_predict = clf.predict(X_CT)

In [ ]:
accuracy_score(y,y_predict)

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 50, p = 3)
knn.fit(X_CT,y)
knn.score(X_CT,y)

In [ ]:
sgd = SGDClassifier(penalty='l1',alpha=0.05)
sgd.fit(X_CT,y)
sgd.score(X_CT,y)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
gbdt = HistGradientBoostingClassifier(learning_rate=0.09)
gbdt.fit(X_CT,y)
gbdt.score(X_CT,y)

# Voting

In [ ]:
clf1 = DecisionTreeClassifier()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = HistGradientBoostingClassifier(learning_rate=0.09)

eclf = VotingClassifier(
    estimators=[('tree', clf1), ('rf', clf2), ('hisb', clf3)],
    voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 
                                                 'histb', 'Ensemble']):
    scores = cross_val_score(clf, X_CT, y, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

f, axes = plt.subplots(1, 4, figsize=(20, 4))
for i,(clf, label) in enumerate(zip([clf1, clf2, clf3, eclf], ['Tree', 
                                                               'Random Forest', 'ADA', 'Ensemble'])):
    y_pred = clf.fit(X_CT, y).predict(X_CT) 
    disp = ConfusionMatrixDisplay(confusion_matrix(y,y_pred))
    disp.plot(ax=axes[i], xticks_rotation=45)
    disp.ax_.set_title(label)
    disp.im_.colorbar.remove()


In [ ]:
# Training classifiers
clf1 = DecisionTreeClassifier()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = GradientBoostingClassifier(learning_rate= 0.95 , subsample=0.75, n_estimators=500)
eclf = VotingClassifier(estimators=[('dt', clf1), ('rf', clf2), ('grad', clf3)],
                        voting='soft', weights=[2, 1, 2])

for clf, label in zip([clf1, clf2, clf3, eclf], ['DecisionTree', 
                                                    'RandomForest', 'GradB', 'Ensemble']):
    scores = cross_val_score(clf, X_CT, y, scoring='accuracy', cv=5)
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


f, axes = plt.subplots(1, 4, figsize=(20, 4))
for i,(clf, label) in enumerate(zip([clf1, clf2, clf3, eclf], ['DecisionTree', 
                                                               'RandomForest', 'GradB', 'Ensemble'])):
    y_pred = clf.fit(X_CT, y).predict(X_CT) 
    disp = ConfusionMatrixDisplay(confusion_matrix(y,y_pred), )
    disp.plot(ax=axes[i], xticks_rotation=45)
    disp.ax_.set_title(label)
    disp.im_.colorbar.remove()


clf1 = clf1.fit(X_CT, y)
clf2 = clf2.fit(X_CT, y)
clf3 = clf3.fit(X_CT, y)
eclf = eclf.fit(X_CT, y)

# Plotting decision regions
f, axarr = plt.subplots(2, 2, sharex="col", sharey="row", figsize=(10, 8))
for idx, clf, tt in zip(
    product([0, 1], [0, 1]),
    [clf1, clf2, clf3, eclf],
    ["Decision Tree ()", "RF (n_est=100)", "Grad", "Soft Voting"],
):
    DecisionBoundaryDisplay.from_estimator(
        clf, X, alpha=0.4, ax=axarr[idx[0], idx[1]], response_method="predict"
    )
    axarr[idx[0], idx[1]].scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor="k")
    axarr[idx[0], idx[1]].set_title(tt)

plt.show()

In [ ]:
df.workclass = df.workclass.replace('Private',0)
df.workclass = df.workclass.replace('Self-Employed',1)
df.workclass = df.workclass.replace('Government',2)
df.workclass = df.workclass.replace('Other',3)

df.occupation = df.occupation.replace('Intellectual labor',0)
df.occupation = df.occupation.replace('Professional',1)
df.occupation = df.occupation.replace('Physical labor',2)
df.occupation = df.occupation.replace('Sales',3)
df.occupation = df.occupation.replace('Other-service',4)

df['marital-status'] = df['marital-status'].replace('Married',0)
df['marital-status'] = df['marital-status'].replace('Single',1)
df['marital-status'] = df['marital-status'].replace('Divorced',2)
df['marital-status'] = df['marital-status'].replace('Widowed',3)
df['marital-status'] = df['marital-status'].replace('Separated',4)

df.sex = df.sex.replace('Male',0)
df.sex = df.sex.replace('Female',1)

df.race = df.race.replace('White',0)
df.race = df.race.replace('Other',1)

df['native-country'] = df['native-country'].replace('United-States',0)
df['native-country'] = df['native-country'].replace('Other',1)

In [ ]:
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_CT, y)
y_pred = forest.predict(X_CT)
print("Accuracy on data: {:.2f}".format(forest.score(X_CT, y)))

plt.figure(figsize=(20,10))
ConfusionMatrixDisplay(confusion_matrix(y,y_pred)).plot();
plt.show();

In [ ]:
y = df['income']
X = df.drop(columns=['income'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
feature_names = X_train.columns
forest_importances = pd.DataFrame(forest.feature_importances_, index=feature_names, columns =['mean'])
forest_importances['std'] = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

sort_forest = forest_importances.sort_values(by='mean')


fig, ax = plt.subplots(figsize=(15,10))
col = sort_forest['mean']
yerr = sort_forest['std']
col.plot.bar(yerr=yerr, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

display(sort_forest.tail(5))

In [ ]:
result = permutation_importance(forest, X_train, y_train, n_repeats=20, random_state=42)

perm_sorted_idx = result.importances_mean.argsort()
tree_importance_sorted_idx = np.argsort(forest.feature_importances_)
tree_indices = np.arange(0, len(forest.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

ax1.barh(tree_indices, forest.feature_importances_[tree_importance_sorted_idx], height=0.7)
ax1.set_yticks(tree_indices)
ax1.set_yticklabels(X_train.columns[tree_importance_sorted_idx])
ax1.set_ylim((0, len(forest.feature_importances_)))
ax2.boxplot(
    result.importances[perm_sorted_idx].T,
    vert=False,
    labels=X_train.columns[perm_sorted_idx],
)
fig.tight_layout()
plt.show()

In [ ]:
from scipy.stats import spearmanr
from scipy.spatial.distance import squareform
from scipy.cluster import hierarchy

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(X_train).correlation

# Ensure the correlation matrix is symmetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels = X_train.columns,
    ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax1.axhline(y=0.8, color='r', linestyle='--')


# distance_matrix = 1 - np.abs(corr)
# dist_linkage = hierarchy.ward(squareform(distance_matrix))
# dendro = hierarchy.dendrogram(
#     dist_linkage, 
#     ax=ax2, leaf_rotation=90
# )

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(X_train.columns[dendro["leaves"]], rotation="vertical")
ax2.set_yticklabels(X_train.columns[dendro["leaves"]])
fig.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict

cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]

X_train_sel = X_train.iloc[:, selected_features]
X_test_sel = X_test.iloc[:, selected_features]

clf_sel = RandomForestClassifier(n_estimators=100, random_state=42)
clf_sel.fit(X_train_sel, y_train)
print(
    "Точность тестовых данных с удаленными признаками: {:.2f}".format(
        clf_sel.score(X_test_sel, y_test)
    )
)
y_pred_train = clf_sel.predict(X_train_sel)
y_pred = clf_sel.predict(X_test_sel)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3))
ConfusionMatrixDisplay(confusion_matrix(y_train,y_pred_train)).plot(ax=ax1);
ConfusionMatrixDisplay(confusion_matrix(y_test,y_pred)).plot(ax=ax2);
plt.show();


selected_features_names = X_train.columns[selected_features]


print(selected_features)
print(selected_features_names)



fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels = X_train.columns,
    ax=ax1, leaf_rotation=90
)

ax1.axhline(y=0.8, color='r', linestyle='--')

[ax1.get_xticklabels()[dendro["leaves"].index(i)].set_color('red') for i  in selected_features];

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(X_train.columns[dendro["leaves"]], rotation="vertical")
ax2.set_yticklabels(X_train.columns[dendro["leaves"]])
fig.tight_layout()
[ax2.get_xticklabels()[dendro["leaves"].index(i)].set_color('red') for i  in selected_features];
[ax2.get_yticklabels()[dendro["leaves"].index(i)].set_color('red') for i  in selected_features];
plt.show()

In [ ]:
result = permutation_importance(clf_sel, X_train_sel, y_train, n_repeats=10, random_state=42)

perm_sorted_idx = result.importances_mean.argsort()
tree_importance_sorted_idx = np.argsort(clf_sel.feature_importances_)
tree_indices = np.arange(0, len(clf_sel.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

ax1.barh(tree_indices, clf_sel.feature_importances_[tree_importance_sorted_idx], height=0.7)
ax1.set_yticks(tree_indices)
ax1.set_yticklabels(selected_features_names[tree_importance_sorted_idx])
ax1.set_ylim((0, len(clf_sel.feature_importances_)))
ax2.boxplot(
    result.importances[perm_sorted_idx].T,
    vert=False,
    labels=selected_features_names[perm_sorted_idx],
)
fig.tight_layout()
plt.show()

In [ ]:
feature_names = selected_features_names[tree_importance_sorted_idx]
forest_importances = pd.DataFrame(clf_sel.feature_importances_, 
                                  index=selected_features_names, 
                                  columns =['mean'])
forest_importances['std'] = np.std([tree.feature_importances_ for tree in clf_sel.estimators_], axis=0)

sort_forest = forest_importances.sort_values(by='mean')


fig, ax = plt.subplots(figsize=(15,10))
col = sort_forest['mean']
yerr = sort_forest['std']
col.plot.bar(yerr=yerr, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

display(sort_forest)

In [ ]:
from sklearn.feature_selection import SelectPercentile


# задаем определенное стартовое значение для воспроизводимости результата
rng = np.random.RandomState(42)
noise = rng.normal(size=(len(X),11 ))
# добавляем к данным шумовые признаки
# первые 30 признаков являются исходными, остальные 50 являются шумовыми
X_w_noise = np.hstack([X, noise])

X_train, X_test, y_train, y_test = train_test_split(X_w_noise, y, random_state = 42, test_size=.5)
# используем f_classif (по умолчанию)
# и SelectPercentile, чтобы выбрать 50% признаков
select_pct = SelectPercentile(percentile=50)
select_pct.fit(X_train, y_train)
# преобразовываем обучающий набор
X_train_selected = select_pct.transform(X_train)

print("форма массива X_train: {}".format(X_train.shape))
print("форма массива X_train_selected: {}".format(X_train_selected.shape))

In [ ]:
mask = select_pct.get_support()
plt.matshow( mask.reshape(1,-1), cmap = 'gray_r', aspect=0.5)
plt.yticks([])
plt.xlabel("Индекс признака");

mask = select_pct.get_support()[:30]
plt.matshow( mask.reshape(1,-1), cmap = 'gray_r', aspect=0.2)
plt.yticks([])
plt.xlabel("Индекс признака");

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train,y_train)

plt.figure(figsize=(15,8))
feature_names = np.array(X_train.columns)
#importance = rfc.feature_importances_

importance = np.sort(rfc.feature_importances_)
importance_name = np.argsort(rfc.feature_importances_)

plt.barh(width=importance, y = feature_names[importance_name] )
ax = plt.gca()
ax.invert_yaxis()
plt.title("Важность признаков")
plt.show()

In [ ]:
importance = rfc.feature_importances_

plt.figure(figsize=(15,8))
plt.barh(width=importance, y=feature_names)
plt.gca().invert_yaxis()
plt.title("Важность признаков")
plt.show()

mask = importance
plt.matshow( mask.reshape(1,-1), cmap = 'Blues', aspect=0.3)
plt.yticks([])
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.xlabel("Индекс признака");

In [ ]:
from sklearn.feature_selection  import RFE

X_train, X_test, y_train, y_test = train_test_split(X_w_noise, y, random_state=42)

rfe_select = RFE(RandomForestClassifier(n_estimators=100, random_state=42),
             n_features_to_select=11)

rfe_select.fit(X_train, y_train)


# визуализируем отобранные признаки:
mask = rfe_select.get_support()
plt.matshow( mask.reshape(1,-1), cmap = 'gray_r', aspect=0.5)
plt.yticks([])
plt.gca().xaxis.set_minor_locator(plt.MultipleLocator(1))
plt.xlabel("Индекс признака");

mask = rfe_select.get_support()[:30]
plt.matshow( mask.reshape(1,-1), cmap = 'gray_r', aspect=0.2)
plt.yticks([])
plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))
plt.xlabel("Индекс признака");

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

select_f = SequentialFeatureSelector(
             RandomForestClassifier(n_estimators=100, random_state=42),
             n_features_to_select = 10 , 
             direction="forward")

select_f.fit(X_train, y_train)
# визуализируем отобранные признаки:
mask = select_f.get_support()
plt.matshow(mask.reshape(1,-1), cmap = 'gray_r', aspect=0.3)
plt.yticks([])
plt.xlabel("Индекс признака");
plt.show();

In [ ]:
rd = RandomForestClassifier(n_estimators=100)
rd.fit(X,y)
rd.score(X,y)

In [ ]:
gbrt = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)
gbrt.fit(X_CT, y)
print('Правильность на обучающем наборе: {:.3f}'.format(gbrt.score(X_CT, y)))

In [ ]:
# отбираю эту модель
forest = RandomForestClassifier(n_estimators=100)
forest.fit(X_CT, y)
y_pred = forest.predict(X_CT)
print("Accuracy on data: {:.2f}".format(forest.score(X_CT, y)))

plt.figure(figsize=(20,10))
ConfusionMatrixDisplay(confusion_matrix(y,y_pred)).plot();
plt.show();

# 3 блок

In [ ]:
path_train = pd.read_csv('income_train.csv')
df = pd.DataFrame(path_train)
df['income'] = np.where(df['income']== '>50K',1,0)
display(df.head(),df.shape)

path_test = pd.read_csv('income_test.csv')
df1 = pd.DataFrame(path_test)
display(df1.head())
df1.shape

## Делаем обработку df (т.е train)

In [ ]:
df = df.replace('?',np.NaN)
df = df.dropna()
# это выброс, тк рядом значений нет, а самое близкое к этому это где-то 43000
df = df.drop(df[df['capital-gain'] == 99999].index)
display(df.head(),df.shape)

In [ ]:
#### Relationship
df.relationship = df.relationship.replace('Husband','Family')
df.relationship = df.relationship.replace('Wife','Family')
df.relationship = df.relationship.replace('Unmarried','Not-in-family')
df.relationship = df.relationship.replace('Own-child','Other')
df.relationship = df.relationship.replace('Other-relative','Other')
###### Occupation
df.occupation = df.occupation.replace('Prof-specialty','Professional')
df.occupation = df.occupation.replace('Craft-repair','Physical labor') # blue - рабочий класс(почасовая полата) - физич труд
df.occupation = df.occupation.replace('Exec-managerial','Intellectual labor') # white - наемный работник(доход в месяц) - умственный
df.occupation = df.occupation.replace('Adm-clerical','Intellectual labor') # white
df.occupation = df.occupation.replace('Machine-op-inspct','Physical labor') # blue
df.occupation = df.occupation.replace('Transport-moving','Physical labor')# blue
df.occupation = df.occupation.replace('Handlers-cleaners','Physical labor') # blue
df.occupation = df.occupation.replace('Farming-fishing','Physical labor')# blue
df.occupation = df.occupation.replace('Tech-support','Other-service')
df.occupation = df.occupation.replace('Protective-serv','Other-service')
df.occupation = df.occupation.replace('Priv-house-serv','Other-service')
df.occupation = df.occupation.replace('Armed-Forces','Other-service')
##### Workclass
df.workclass = df.workclass.replace('Federal-gov','Government')
df.workclass = df.workclass.replace('State-gov','Government')
df.workclass = df.workclass.replace('Local-gov','Government')
df.workclass = df.workclass.replace('Self-emp-inc','Self-Employed')
df.workclass = df.workclass.replace('Self-emp-not-inc','Self-Employed')
df.workclass = df.workclass.replace('Never-worked', 'Other')
df.workclass = df.workclass.replace('Without-pay', 'Other')
#### Marital
df['marital-status'] = df['marital-status'].replace('Married-civ-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-AF-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-spouse-absent','Married')
df['marital-status'] = df['marital-status'].replace('Never-married','Single')
##### Race
df.race = df.race.replace('Black','Other')
df.race = df.race.replace('Asian-Pac-Islander','Other')
df.race = df.race.replace('Amer-Indian-Eskimo','Other')
##### Country
country = df['native-country'].unique()
def zamena_country(data):
    for i in country:
        if i != 'United-States':
            data['native-country'] = data['native-country'].replace(i,'Other')
    return data
df = zamena_country(df)

###### Capital changes
df['capital-changes']  = df['capital-gain'] - df['capital-loss']
##### 
df = df.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
df.head()

## Column Transformer

In [ ]:
num_features = ['age','hours-per-week','capital-changes']
y = np.array(df.income)
X = df.drop(columns=['income'])

In [ ]:
workclass_map = [{
    'col':'workclass', 
    'mapping':{ 'Other':3, 'Government':2, 'Self-Employed':1, 'Private':0}
    }]
occupation_map = [{
    'col':'occupation',
    'mapping':{'Other-service':4, 'Sales':3,'Physical labor':2, 'Professional':1, 'Intellectual labor':0}
    }]
marital_map = [{
    'col':'marital-status',
    'mapping':{'Separated': 4, 'Widowed': 3, 'Divorced':2, 'Single': 1, 'Married':0}
    }]
sex_map = [{
    'col':'sex',
    'mapping':{'Female':1, 'Male':0}
    }]
race_map = [{
    'col':'race',
    'mapping':{'Other':1, 'White':0}
    }]
native_country_map = [{
    'col':'native-country',
    'mapping':{'Other':1, 'United-States':0}
    }]
education_map = [{
    'col':'education',
    'mapping':{'Academia':2, 'School':1, 'Higher-educ':0}
    }]
relationship_map = [{
    'col':'relationship',
    'mapping':{'Other':2, 'Not-in-family':1, 'Family':0}
    }]

sex_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = sex_map))])

workclass_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = workclass_map))])

marital_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = marital_map))])

occupation_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = occupation_map))])

race_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = race_map))])

native_country_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = native_country_map))])
education_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = education_map))])
relationship_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = relationship_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('sex', sex_transformer, ['sex']),
                                     ('work', workclass_transformer, ['workclass']),
                                     ('matiral', marital_transformer, ['marital-status']),
                                     ('occupation', occupation_transformer, ['occupation']),
                                     ('race', race_transformer, ['race']),
                                     ('native-country',native_country_transformer, ['native-country']),
                                     ('relationship',relationship_transformer,['relationship'])],
                                      remainder = 'passthrough')

display(CT)

### Делаем обработку df1 (т.е test)

In [ ]:
df1.describe()

In [ ]:
ax = sns.pairplot(df1, palette= 'inferno')

In [ ]:
ax = sns.regplot(x = 'age' , y = 'capital-gain', data = df1, fit_reg= True, scatter_kws= {'color':'red'}, line_kws={'color':'black'})

In [ ]:
display(df1[df1['capital-gain'] == 99999])

In [ ]:
df1 = df1.replace('?',np.NaN)
df1 = df1.dropna()
# это выброс, тк рядом значений нет, а самое близкое к этому это где-то 43000
df1 = df1.drop(df1[df1['capital-gain'] == 99999].index)
df1.head()

In [ ]:
#### Relationship
df1.relationship = df1.relationship.replace('Husband','Family')
df1.relationship = df1.relationship.replace('Wife','Family')
df1.relationship = df1.relationship.replace('Unmarried','Not-in-family')
df1.relationship = df1.relationship.replace('Own-child','Other')
df1.relationship = df1.relationship.replace('Other-relative','Other')
###### Occupation
df1.occupation = df1.occupation.replace('Prof-specialty','Professional')
df1.occupation = df1.occupation.replace('Craft-repair','Physical labor') # blue - рабочий класс(почасовая полата) - физич труд
df1.occupation = df1.occupation.replace('Exec-managerial','Intellectual labor') # white - наемный работник(доход в месяц) - умственный
df1.occupation = df1.occupation.replace('Adm-clerical','Intellectual labor') # white
df1.occupation = df1.occupation.replace('Machine-op-inspct','Physical labor') # blue
df1.occupation = df1.occupation.replace('Transport-moving','Physical labor')# blue
df1.occupation = df1.occupation.replace('Handlers-cleaners','Physical labor') # blue
df1.occupation = df1.occupation.replace('Farming-fishing','Physical labor')# blue
df1.occupation = df1.occupation.replace('Tech-support','Other-service')
df1.occupation = df1.occupation.replace('Protective-serv','Other-service')
df1.occupation = df1.occupation.replace('Priv-house-serv','Other-service')
df1.occupation = df1.occupation.replace('Armed-Forces','Other-service')
##### Workclass
df1.workclass = df1.workclass.replace('Federal-gov','Government')
df1.workclass = df1.workclass.replace('State-gov','Government')
df1.workclass = df1.workclass.replace('Local-gov','Government')
df1.workclass = df1.workclass.replace('Self-emp-inc','Self-Employed')
df1.workclass = df1.workclass.replace('Self-emp-not-inc','Self-Employed')
df1.workclass = df1.workclass.replace('Never-worked', 'Other')
df1.workclass = df1.workclass.replace('Without-pay', 'Other')
#### Marital
df1['marital-status'] = df1['marital-status'].replace('Married-civ-spouse','Married')
df1['marital-status'] = df1['marital-status'].replace('Married-AF-spouse','Married')
df1['marital-status'] = df1['marital-status'].replace('Married-spouse-absent','Married')
df1['marital-status'] = df1['marital-status'].replace('Unmarried','Single')
##### Race
df1.race = df1.race.replace('Black','Other')
df1.race = df1.race.replace('Asian-Pac-Islander','Other')
df1.race = df1.race.replace('Amer-Indian-Eskimo','Other')
##### Country
country = df1['native-country'].unique()
def zamena_country(data):
    for i in country:
        if i != 'United-States':
            data['native-country'] = data['native-country'].replace(i,'Other')
    return data
df1 = zamena_country(df1)

###### Capital changes
df1['capital-changes']  = df1['capital-gain'] - df1['capital-loss']
##### 
df1 = df1.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
df1.head()

In [ ]:
# делаем маску
mask = df1.index

In [ ]:
finish_pipe_RFR =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestClassifier(n_estimators=100))
            ])

finish_pipe_RFR.fit(X, y)
print(finish_pipe_RFR.score(X,y))
print(classification_report(y, finish_pipe_RFR.predict(X)))
print('Правильность на наборе: {:.3f}'.format(finish_pipe_RFR.score(X, y))) 

In [ ]:
y_predict = finish_pipe_RFR.predict(df1)
y_predict.shape